In [48]:
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text

In [5]:
df = pd.read_csv('spam.csv')

In [9]:
df

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will ü b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


In [13]:
df.groupby('Category').describe()

Message                                                            \
           count unique                                                top   
Category                                                                     
ham         4825   4516                             Sorry, I'll call later   
spam         747    641  Please call our customer service representativ...   

               
         freq  
Category       
ham        30  
spam        4

In [ ]:
#Converting the imbalanced dataset to balanced dataset

In [16]:
df_spam = df[df['Category']=='spam']
df_spam.shape

(747, 2)

In [17]:
df_ham = df[df['Category']=='ham']
df_ham.shape

(4825, 2)

In [18]:
#Down sampling

df_downsample = df_ham.sample(df_spam.shape[0])
df_downsample.shape

(747, 2)

In [19]:
df_balanced = pd.concat([df_spam,df_downsample])
df_balanced.groupby('Category').describe()

Message                                                            \
           count unique                                                top   
Category                                                                     
ham          747    735                             Sorry, I'll call later   
spam         747    641  Please call our customer service representativ...   

               
         freq  
Category       
ham         4  
spam        4

In [20]:
df_balanced.head(5)

,Category,Message
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
5,spam,FreeMsg Hey there darling it's been 3 week's n...
8,spam,WINNER!! As a valued network customer you have...
9,spam,Had your mobile 11 months or more? U R entitle...
11,spam,"SIX chances to win CASH! From 100 to 20,000 po..."


In [28]:
# encoding the text category column
df_balanced['spam'] = pd.get_dummies(df_balanced.Category, drop_first=True)

In [29]:
df_balanced.head()

,Category,Message,spam
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,1
5,spam,FreeMsg Hey there darling it's been 3 week's n...,1
8,spam,WINNER!! As a valued network customer you have...,1
9,spam,Had your mobile 11 months or more? U R entitle...,1
11,spam,"SIX chances to win CASH! From 100 to 20,000 po...",1


In [30]:
df_balanced.drop(['Category'], axis=1, inplace=True)

In [31]:
df_balanced.head()

,Message,spam
2,Free entry in 2 a wkly comp to win FA Cup fina...,1
5,FreeMsg Hey there darling it's been 3 week's n...,1
8,WINNER!! As a valued network customer you have...,1
9,Had your mobile 11 months or more? U R entitle...,1
11,"SIX chances to win CASH! From 100 to 20,000 po...",1


In [33]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train, y_test = train_test_split(df_balanced['Message'],df_balanced['spam'], stratify=df_balanced['spam'])

In [36]:
X_train.shape

(1120,)

In [2]:
preprocess_url = 'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3' 
encoder_url = 'https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4'

In [38]:
bert_preprocess = hub.KerasLayer(preprocess_url)
bert_encoder = hub.KerasLayer(encoder_url)

In [39]:
def get_sentence_embedding(Sentences):
    preprocessed_text = bert_preprocess(Sentences)
    return bert_encoder(preprocessed_text)['pooled_output']

In [40]:
e= get_sentence_embedding(
[
    'Grapes',
    'Banana',
    'Jeff bezos',
    'Bill Gates'
])


In [41]:
e[0]

<tf.Tensor: shape=(768,), dtype=float32, numpy=
array([-8.60232115e-01, -2.12429434e-01,  4.91568863e-01,  6.92563057e-01,
       -3.30052406e-01, -1.40575528e-01,  9.22929645e-01,  1.98044851e-01,
        1.72779053e-01, -9.99786615e-01,  1.84800904e-02,  2.22860381e-01,
        9.63335872e-01, -1.37402520e-01,  9.09067273e-01, -6.16936028e-01,
       -2.00021833e-01, -5.61885417e-01,  4.16765541e-01, -8.19490969e-01,
        5.30651808e-01,  8.38440180e-01,  4.94481474e-01,  3.06373864e-01,
        3.90681058e-01,  2.38232911e-01, -5.68000019e-01,  9.01755154e-01,
        9.34361815e-01,  6.79499626e-01, -6.92578197e-01,  1.88730538e-01,
       -9.68373597e-01, -2.02293187e-01,  4.33047324e-01, -9.70722914e-01,
        1.85543060e-01, -7.50942826e-01, -6.77514896e-02, -2.69225575e-02,
       -8.55486751e-01,  2.66132712e-01,  9.94937241e-01,  1.47690877e-01,
        1.83297638e-02, -3.44092488e-01, -9.99768496e-01,  2.55884081e-01,
       -8.28456879e-01, -2.80603021e-01, -1.87773049

In [46]:
from sklearn.metrics.pairwise import cosine_similarity

cosine_similarity([e[3]],[e[2]])

array([[0.8759414]], dtype=float32)

In [49]:
#Build model
# Bert layers
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
preprocessed_text = bert_preprocess(text_input)
outputs = bert_encoder(preprocessed_text)

# Neural network layers
l = tf.keras.layers.Dropout(0.1, name="dropout")(outputs['pooled_output'])
l = tf.keras.layers.Dense(1, activation='sigmoid', name="output")(l)

# Use inputs and outputs to construct a final model
model = tf.keras.Model(inputs=[text_input], outputs = [l])

In [50]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 text (InputLayer)              [(None,)]            0           []                               
                                                                                                  
 keras_layer_3 (KerasLayer)     {'input_type_ids':   0           ['text[0][0]']                   
                                (None, 128),                                                      
                                 'input_word_ids':                                                
                                (None, 128),                                                      
                                 'input_mask': (Non                                               
                                e, 128)}                                                      

In [51]:
len(X_train)

1120

In [52]:
METRICS = [
      tf.keras.metrics.BinaryAccuracy(name='accuracy'),
      tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Recall(name='recall')
]

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=METRICS)

In [53]:
# Model training
model.fit(X_train, y_train, epochs=10)

Epoch 1/10
35/35 [==============================] - 249s 7s/step - loss: 0.6631 - accuracy: 0.6089 - precision: 0.6121 - recall: 0.5946
Epoch 2/10
35/35 [==============================] - 247s 7s/step - loss: 0.5390 - accuracy: 0.7670 - precision: 0.7627 - recall: 0.7750
Epoch 3/10
35/35 [==============================] - 242s 7s/step - loss: 0.4644 - accuracy: 0.8482 - precision: 0.8385 - recall: 0.8625
Epoch 4/10
35/35 [==============================] - 235s 7s/step - loss: 0.4110 - accuracy: 0.8580 - precision: 0.8487 - recall: 0.8714
Epoch 5/10
35/35 [==============================] - 250s 7s/step - loss: 0.3781 - accuracy: 0.8875 - precision: 0.8767 - recall: 0.9018
Epoch 6/10
35/35 [==============================] - 293s 8s/step - loss: 0.3534 - accuracy: 0.8955 - precision: 0.8866 - recall: 0.9071
Epoch 7/10
35/35 [==============================] - 271s 8s/step - loss: 0.3329 - accuracy: 0.8982 - precision: 0.8899 - recall: 0.9089
Epoch 8/10
35/35 [==============================

In [ ]:
model.evaluate(X_test, y_test)

In [ ]:
y_predicted = model.predict(X_test)
y_predicted = y_predicted.flatten()

In [ ]:
import numpy as np

y_predicted = np.where(y_predicted > 0.5, 1, 0)
y_predicted

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report

cm = confusion_matrix(y_test, y_predicted)
cm 

In [ ]:
from matplotlib import pyplot as plt
import seaborn as sn
sn.heatmap(cm, annot=True, fmt='d')
plt.xlabel('Predicted')
plt.ylabel('Truth')

In [ ]:
print(classification_report(y_test, y_predicted))

In [ ]:
reviews = [
    'Enter a chance to win $5000, hurry up, offer valid until march 31, 2021',
    'You are awarded a SiPix Digital Camera! call 09061221061 from landline. Delivery within 28days. T Cs Box177. M221BP. 2yr warranty. 150ppm. 16 . p pÂ£3.99',
    'it to 80488. Your 500 free text messages are valid until 31 December 2005.',
    'Hey Sam, Are you coming for a cricket game tomorrow',
    "Why don't you wait 'til at least wednesday to see if you get your ."
]
model.predict(reviews)